In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

In [50]:
data = pd.read_csv('./database/mazda2.csv')
car_data = data.dropna(axis = 1)
car_data['age'] = car_data.year.apply(lambda x: 2020 - x)
car_data.drop(axis=1, columns = ['Unnamed: 0', 'year','url', 'id'], inplace = True)
car_data

/Users/minaevpavel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/minaevpavel/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,brand,model,mileage,body_type,fuel_type,engine_volume,drive,price,engine_power,gearbox,age
0,Mazda,CX-7,180000,внедорожник,бензин,2.3,полный,615000,238.0,AT,9
1,Mazda,CX-7,128000,внедорожник,бензин,2.3,полный,690000,238.0,AT,9
2,Mazda,6,283889,седан,бензин,1.8,передний,199990,120.0,MT,18
3,Mazda,6,100000,седан,бензин,1.8,передний,610000,120.0,MT,9
4,Mazda,6,39761,седан,бензин,2.5,передний,1630000,194.0,AT,2
...,...,...,...,...,...,...,...,...,...,...,...
4769,Mazda,6,127855,хетчбэк,бензин,2.0,передний,715000,147.0,AT,8
4770,Mazda,6,145000,седан,бензин,2.0,передний,1100000,150.0,AT,4
4771,Mazda,6,95000,седан,бензин,2.0,передний,1000000,150.0,AT,5
4772,Mazda,6,44000,седан,бензин,2.0,передний,990000,150.0,AT,6


In [51]:
distinct_cars = car_data.drop(axis=1, columns = ['age', 'mileage','price'])
distinct_cars.drop_duplicates(inplace = True, ignore_index = True)
distinct_cars['model_id'] = pd.Series(data = list(range(distinct_cars.shape[0])))
distinct_cars

,brand,model,body_type,fuel_type,engine_volume,drive,engine_power,gearbox,model_id
0,Mazda,CX-7,внедорожник,бензин,2.3,полный,238.0,AT,0
1,Mazda,6,седан,бензин,1.8,передний,120.0,MT,1
2,Mazda,6,седан,бензин,2.5,передний,194.0,AT,2
3,Mazda,3,хетчбэк,бензин,1.6,передний,105.0,MT,3
4,Mazda,CX-5,внедорожник,бензин,2.0,полный,150.0,AT,4
...,...,...,...,...,...,...,...,...,...
350,Mazda,626,седан,дизель,2.0,передний,101.0,MT,350
351,Mazda,MX-5,кабриолет,бензин,2.0,задний,160.0,MT,351
352,Mazda,929,купе,бензин,2.0,задний,90.0,MT,352
353,Mazda,Premacy,минивэн,бензин,2.0,передний,151.0,AT,353


In [54]:
join_on = list(distinct_cars.columns)[:-1]
merged_data = car_data.merge(distinct_cars, on=join_on)
merged_data

,brand,model,mileage,body_type,fuel_type,engine_volume,drive,price,engine_power,gearbox,age,model_id
0,Mazda,CX-7,180000,внедорожник,бензин,2.3,полный,615000,238.0,AT,9,0
1,Mazda,CX-7,128000,внедорожник,бензин,2.3,полный,690000,238.0,AT,9,0
2,Mazda,CX-7,116300,внедорожник,бензин,2.3,полный,575000,238.0,AT,9,0
3,Mazda,CX-7,164852,внедорожник,бензин,2.3,полный,375000,238.0,AT,13,0
4,Mazda,CX-7,114712,внедорожник,бензин,2.3,полный,750000,238.0,AT,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4769,Mazda,626,150000,седан,дизель,2.0,передний,5000,101.0,MT,21,350
4770,Mazda,MX-5,108000,кабриолет,бензин,2.0,задний,600000,160.0,MT,13,351
4771,Mazda,929,150000,купе,бензин,2.0,задний,80000,90.0,MT,33,352
4772,Mazda,Premacy,160000,минивэн,бензин,2.0,передний,520000,151.0,AT,11,353


In [55]:
merged_data['counts'] = merged_data.model_id.apply(lambda x: merged_data.model_id.value_counts()[x])
merged_data.drop(axis = 0, index = merged_data[merged_data.counts <= 10].index, inplace = True)
merged_data

,brand,model,mileage,body_type,fuel_type,engine_volume,drive,price,engine_power,gearbox,age,model_id,counts
0,Mazda,CX-7,180000,внедорожник,бензин,2.3,полный,615000,238.0,AT,9,0,460
1,Mazda,CX-7,128000,внедорожник,бензин,2.3,полный,690000,238.0,AT,9,0,460
2,Mazda,CX-7,116300,внедорожник,бензин,2.3,полный,575000,238.0,AT,9,0,460
3,Mazda,CX-7,164852,внедорожник,бензин,2.3,полный,375000,238.0,AT,13,0,460
4,Mazda,CX-7,114712,внедорожник,бензин,2.3,полный,750000,238.0,AT,8,0,460
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4504,Mazda,6,293000,седан,бензин,2.3,передний,200000,166.0,MT,17,202,18
4505,Mazda,6,260000,седан,бензин,2.3,передний,190000,166.0,MT,16,202,18
4506,Mazda,6,250000,седан,бензин,2.3,передний,220000,166.0,MT,17,202,18
4507,Mazda,6,215000,седан,бензин,2.3,передний,225000,166.0,MT,16,202,18


In [62]:
X = merged_data.drop(axis = 1, columns = ['brand',
 'model',
 'body_type',
 'fuel_type',
 'engine_volume',
 'drive',
 'engine_power',
 'gearbox',
 'counts'])
y = X.price
X.drop(axis = 1, columns = ['price'], inplace = True)

In [64]:
 X_train, X_test, y_train, y_test = train_test_split(  X, y, test_size=0.33, random_state=42)

In [71]:
X_train

,mileage,age,model_id
945,197000,16,3
462,211736,14,1
2386,110000,10,21
328,75000,10,0
1672,185000,14,12
...,...,...,...
1130,47000,4,4
1294,114800,6,8
860,160200,13,3
3606,109650,5,58


In [70]:
regressor = CatBoostRegressor(learning_rate=1, depth=6, loss_function='RMSE')
regressor.fit(X_train, y_train, cat_features = ['model_id']);
y_pred = regressor.predict(X_test)
mean_absolute_error(y_test, y_pred )